In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils


In [2]:
% load_ext autoreload 
% autoreload 2

In [3]:
from modules_tied import VAE
from modules_tied import NetD
from modules_tied import Dummy
from modules_tied import loss_function

In [4]:
bsz = 100
criterion = nn.BCELoss()


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)


In [6]:
netG = VAE()
netD = NetD()
dummy = Dummy()

In [7]:
optimizerD = optim.Adam(netD.parameters(), lr=1e-4)
optimizerG = optim.Adam(netG.parameters(), lr=1e-4)
optimizer_dummy = optim.Adam(dummy.parameters(), lr=1e-4)

In [8]:
input = torch.FloatTensor(bsz,28,28)
label = torch.FloatTensor(bsz)
real_label=1
fake_label=0
USE_CUDA=1

if(USE_CUDA):
    netG=netG.cuda()
    netD=netD.cuda()
    dummy=dummy.cuda()
    criterion=criterion.cuda()
    input,label=input.cuda(), label.cuda()

In [9]:
from modules_tied import Dummy

In [ ]:
for epoch in range(10000):
    for i, (data,_) in enumerate(train_loader):
        #update D: maximize log((D(x)) + log(1-D(G(z)))
        netD.zero_grad()
        dummy.zero_grad()
        
        real_cpu = data
        
        if(USE_CUDA):
            real_cpu = real_cpu.cuda()

        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(bsz).fill_(real_label)

        inputv = Variable(input)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        #train with fake
        fake,mu,logvar = netG(inputv)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()

        zd = torch.FloatTensor(bsz,20).normal_(0,1).cuda()
        zd = Variable(zd)
        x_dummy = dummy(zd,netG)
        out_dummy = netD(x_dummy.detach())
        L_dummy = criterion(out_dummy, labelv)
        L_dummy.backward()
        
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake + L_dummy
        optimizerD.step()

        #step dummy
        x_dummy = dummy(zd,netG)
        out_dummy = netD(x_dummy.detach())
        labelv = Variable(label.fill_(real_label))
        LGAN_dummy = criterion(out_dummy,labelv)
        LGAN_dummy.backward()
        out_orig = netD(inputv)
        L_recon_dummy = loss_function(out_dummy.detach(),out_orig,mu.detach(),logvar.detach())
        L_recon_dummy.backward()
        optimizer_dummy.step()
        
        
        # update G: maximize log((D(G(z))
        if epoch%1==0:
            netG.zero_grad()
            labelv = Variable(label.fill_(real_label))
            output = netD(fake)
            output_orig = netD(inputv)
            #errG = criterion(output,labelv) + loss_function(fake,inputv,mu,logvar)
            #errG = loss_function(output,output_orig,mu,logvar) #+ 0.5*criterion(output,labelv)
            errG = loss_function(fake,inputv,mu,logvar) +loss_function(output,output_orig,mu,logvar)

            errG.backward()
    
            D_G_z2 = output.data.mean()
            optimizerG.step()

        if(epoch %1 ==0):
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                  % (epoch, 100, i, len(train_loader),
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
            if i % 100 == 0:
                print('real_cpu.size()', real_cpu.size())
                vutils.save_image(real_cpu,
                                  './real_samples.png',
                                  normalize=True)
                vutils.save_image(fake.data.view(-1,1,28,28),
                                  './fake_samples.png',
                                  normalize=True)


/home/placeholder/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1, 1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[0/100][0/600] Loss_D: 0.0204 Loss_G: 1.1103 D(x): 0.9854 D(G(z)): 0.0043 / 0.0043
real_cpu.size() torch.Size([100, 1, 28, 28])
[0/100][1/600] Loss_D: 0.0095 Loss_G: 1.1248 D(x): 0.9936 D(G(z)): 0.0029 / 0.0031
[0/100][2/600] Loss_D: 0.0162 Loss_G: 1.1166 D(x): 0.9924 D(G(z)): 0.0083 / 0.0086
[0/100][3/600] Loss_D: 0.0323 Loss_G: 1.0960 D(x): 0.9817 D(G(z)): 0.0110 / 0.0120
[0/100][4/600] Loss_D: 0.0140 Loss_G: 1.1230 D(x): 0.9961 D(G(z)): 0.0096 / 0.0096
[0/100][5/600] Loss_D: 0.0181 Loss_G: 1.1077 D(x): 0.9927 D(G(z)): 0.0104 / 0.0101
[0/100][6/600] Loss_D: 0.0122 Loss_G: 1.1221 D(x): 0.9951 D(G(z)): 0.0072 / 0.0068
[0/100][7/600] Loss_D: 0.0104 Loss_G: 1.1274 D(x): 0.9963 D(G(z)): 0.0066 / 0.0061
[0/100][8/600] Loss_D: 0.0172 Loss_G: 1.1110 D(x): 0.9894 D(G(z)): 0.0058 / 0.0058
[0/100][9/600] Loss_D: 0.0193 Loss_G: 1.1098 D(x): 0.9915 D(G(z)): 0.0101 / 0.0094
[0/100][10/600] Loss_D: 0.0167 Loss_G: 1.1165 D(x): 0.9898 D(G(z)): 0.0062 / 0.0061
[0/100][11/600] Loss_D: 0.0175 Loss_G: 1.

[0/100][99/600] Loss_D: 0.0131 Loss_G: 1.0934 D(x): 0.9957 D(G(z)): 0.0085 / 0.0078
[0/100][100/600] Loss_D: 0.0102 Loss_G: 1.0956 D(x): 0.9976 D(G(z)): 0.0075 / 0.0065
real_cpu.size() torch.Size([100, 1, 28, 28])
[0/100][101/600] Loss_D: 0.0121 Loss_G: 1.0966 D(x): 0.9904 D(G(z)): 0.0016 / 0.0016
[0/100][102/600] Loss_D: 0.0125 Loss_G: 1.0960 D(x): 0.9928 D(G(z)): 0.0050 / 0.0050
[0/100][103/600] Loss_D: 0.0113 Loss_G: 1.0936 D(x): 0.9928 D(G(z)): 0.0035 / 0.0036
[0/100][104/600] Loss_D: 0.0113 Loss_G: 1.0914 D(x): 0.9945 D(G(z)): 0.0057 / 0.0058
[0/100][105/600] Loss_D: 0.0120 Loss_G: 1.0946 D(x): 0.9938 D(G(z)): 0.0055 / 0.0055
[0/100][106/600] Loss_D: 0.0108 Loss_G: 1.0997 D(x): 0.9960 D(G(z)): 0.0067 / 0.0066
[0/100][107/600] Loss_D: 0.0093 Loss_G: 1.0941 D(x): 0.9954 D(G(z)): 0.0045 / 0.0044
[0/100][108/600] Loss_D: 0.0103 Loss_G: 1.1007 D(x): 0.9975 D(G(z)): 0.0075 / 0.0070
[0/100][109/600] Loss_D: 0.0140 Loss_G: 1.0910 D(x): 0.9925 D(G(z)): 0.0062 / 0.0058
[0/100][110/600] Loss

[0/100][197/600] Loss_D: 0.0062 Loss_G: 1.0990 D(x): 0.9984 D(G(z)): 0.0045 / 0.0042
[0/100][198/600] Loss_D: 0.0043 Loss_G: 1.0996 D(x): 0.9982 D(G(z)): 0.0025 / 0.0023
[0/100][199/600] Loss_D: 0.0052 Loss_G: 1.0967 D(x): 0.9985 D(G(z)): 0.0036 / 0.0033
[0/100][200/600] Loss_D: 0.0057 Loss_G: 1.0961 D(x): 0.9967 D(G(z)): 0.0023 / 0.0021
real_cpu.size() torch.Size([100, 1, 28, 28])
[0/100][201/600] Loss_D: 0.0070 Loss_G: 1.0856 D(x): 0.9951 D(G(z)): 0.0020 / 0.0019
[0/100][202/600] Loss_D: 0.0044 Loss_G: 1.0973 D(x): 0.9974 D(G(z)): 0.0016 / 0.0016
[0/100][203/600] Loss_D: 0.0056 Loss_G: 1.0968 D(x): 0.9967 D(G(z)): 0.0021 / 0.0021
[0/100][204/600] Loss_D: 0.0084 Loss_G: 1.0870 D(x): 0.9947 D(G(z)): 0.0029 / 0.0029
[0/100][205/600] Loss_D: 0.0116 Loss_G: 1.0802 D(x): 0.9914 D(G(z)): 0.0026 / 0.0028
[0/100][206/600] Loss_D: 0.0094 Loss_G: 1.0836 D(x): 0.9959 D(G(z)): 0.0051 / 0.0054
[0/100][207/600] Loss_D: 0.0070 Loss_G: 1.0908 D(x): 0.9973 D(G(z)): 0.0042 / 0.0043
[0/100][208/600] Los

[0/100][296/600] Loss_D: 0.0176 Loss_G: 1.0817 D(x): 0.9987 D(G(z)): 0.0153 / 0.0119
[0/100][297/600] Loss_D: 0.0186 Loss_G: 1.0768 D(x): 0.9945 D(G(z)): 0.0123 / 0.0094
[0/100][298/600] Loss_D: 0.0145 Loss_G: 1.0804 D(x): 0.9948 D(G(z)): 0.0086 / 0.0065
[0/100][299/600] Loss_D: 0.0125 Loss_G: 1.0834 D(x): 0.9948 D(G(z)): 0.0069 / 0.0051
[0/100][300/600] Loss_D: 0.0151 Loss_G: 1.0828 D(x): 0.9905 D(G(z)): 0.0050 / 0.0040
real_cpu.size() torch.Size([100, 1, 28, 28])
[0/100][301/600] Loss_D: 0.0094 Loss_G: 1.0890 D(x): 0.9964 D(G(z)): 0.0056 / 0.0043
[0/100][302/600] Loss_D: 0.0107 Loss_G: 1.0881 D(x): 0.9930 D(G(z)): 0.0034 / 0.0028
[0/100][303/600] Loss_D: 0.0089 Loss_G: 1.0920 D(x): 0.9974 D(G(z)): 0.0060 / 0.0049
[0/100][304/600] Loss_D: 0.0089 Loss_G: 1.0854 D(x): 0.9936 D(G(z)): 0.0024 / 0.0020
[0/100][305/600] Loss_D: 0.0217 Loss_G: 1.0689 D(x): 0.9848 D(G(z)): 0.0056 / 0.0054
[0/100][306/600] Loss_D: 0.0073 Loss_G: 1.0997 D(x): 0.9957 D(G(z)): 0.0029 / 0.0028
[0/100][307/600] Los

...

[12/100][134/600] Loss_D: 0.0254 Loss_G: 1.0771 D(x): 0.9887 D(G(z)): 0.0135 / 0.0118
[12/100][135/600] Loss_D: 0.0157 Loss_G: 1.0990 D(x): 0.9908 D(G(z)): 0.0063 / 0.0058
[12/100][136/600] Loss_D: 0.0151 Loss_G: 1.0991 D(x): 0.9908 D(G(z)): 0.0057 / 0.0054
[12/100][137/600] Loss_D: 0.0144 Loss_G: 1.1018 D(x): 0.9922 D(G(z)): 0.0064 / 0.0062
[12/100][138/600] Loss_D: 0.0188 Loss_G: 1.0894 D(x): 0.9882 D(G(z)): 0.0067 / 0.0069
[12/100][139/600] Loss_D: 0.0349 Loss_G: 1.0685 D(x): 0.9749 D(G(z)): 0.0084 / 0.0103
[12/100][140/600] Loss_D: 0.0213 Loss_G: 1.0888 D(x): 0.9890 D(G(z)): 0.0100 / 0.0123
[12/100][141/600] Loss_D: 0.0349 Loss_G: 1.0606 D(x): 0.9799 D(G(z)): 0.0130 / 0.0168
[12/100][142/600] Loss_D: 0.0318 Loss_G: 1.0736 D(x): 0.9962 D(G(z)): 0.0267 / 0.0255
[12/100][143/600] Loss_D: 0.0376 Loss_G: 1.0779 D(x): 0.9963 D(G(z)): 0.0323 / 0.0214
[12/100][144/600] Loss_D: 0.0209 Loss_G: 1.0867 D(x): 0.9916 D(G(z)): 0.0122 / 0.0079
[12/100][145/600] Loss_D: 0.0191 Loss_G: 1.0860 D(x): 

[12/100][229/600] Loss_D: 0.0175 Loss_G: 1.0939 D(x): 0.9922 D(G(z)): 0.0094 / 0.0092
[12/100][230/600] Loss_D: 0.0123 Loss_G: 1.1039 D(x): 0.9951 D(G(z)): 0.0071 / 0.0069
[12/100][231/600] Loss_D: 0.0216 Loss_G: 1.0837 D(x): 0.9874 D(G(z)): 0.0086 / 0.0088
[12/100][232/600] Loss_D: 0.0347 Loss_G: 1.0665 D(x): 0.9784 D(G(z)): 0.0106 / 0.0122
[12/100][233/600] Loss_D: 0.0138 Loss_G: 1.1017 D(x): 0.9921 D(G(z)): 0.0057 / 0.0066
[12/100][234/600] Loss_D: 0.0239 Loss_G: 1.0829 D(x): 0.9891 D(G(z)): 0.0117 / 0.0131
[12/100][235/600] Loss_D: 0.0139 Loss_G: 1.0980 D(x): 0.9946 D(G(z)): 0.0083 / 0.0088
[12/100][236/600] Loss_D: 0.0212 Loss_G: 1.0871 D(x): 0.9961 D(G(z)): 0.0170 / 0.0150
[12/100][237/600] Loss_D: 0.0209 Loss_G: 1.0912 D(x): 0.9924 D(G(z)): 0.0129 / 0.0107
[12/100][238/600] Loss_D: 0.0279 Loss_G: 1.0803 D(x): 0.9863 D(G(z)): 0.0123 / 0.0103
[12/100][239/600] Loss_D: 0.0204 Loss_G: 1.0864 D(x): 0.9922 D(G(z)): 0.0123 / 0.0103
[12/100][240/600] Loss_D: 0.0284 Loss_G: 1.0697 D(x): 

[12/100][328/600] Loss_D: 0.0420 Loss_G: 1.0589 D(x): 0.9679 D(G(z)): 0.0073 / 0.0117
[12/100][329/600] Loss_D: 0.0211 Loss_G: 1.0803 D(x): 0.9938 D(G(z)): 0.0143 / 0.0192
[12/100][330/600] Loss_D: 0.0323 Loss_G: 1.0601 D(x): 0.9867 D(G(z)): 0.0180 / 0.0216
[12/100][331/600] Loss_D: 0.0141 Loss_G: 1.0899 D(x): 0.9930 D(G(z)): 0.0069 / 0.0081
[12/100][332/600] Loss_D: 0.0282 Loss_G: 1.0807 D(x): 0.9961 D(G(z)): 0.0231 / 0.0206
[12/100][333/600] Loss_D: 0.0152 Loss_G: 1.0987 D(x): 0.9970 D(G(z)): 0.0119 / 0.0095
[12/100][334/600] Loss_D: 0.0182 Loss_G: 1.0896 D(x): 0.9909 D(G(z)): 0.0088 / 0.0072
[12/100][335/600] Loss_D: 0.0094 Loss_G: 1.1017 D(x): 0.9940 D(G(z)): 0.0033 / 0.0028
[12/100][336/600] Loss_D: 0.0178 Loss_G: 1.0917 D(x): 0.9944 D(G(z)): 0.0120 / 0.0093
[12/100][337/600] Loss_D: 0.0117 Loss_G: 1.1027 D(x): 0.9928 D(G(z)): 0.0043 / 0.0036
[12/100][338/600] Loss_D: 0.0237 Loss_G: 1.0799 D(x): 0.9834 D(G(z)): 0.0064 / 0.0063
[12/100][339/600] Loss_D: 0.0130 Loss_G: 1.1023 D(x): 

[12/100][427/600] Loss_D: 0.0145 Loss_G: 1.0921 D(x): 0.9932 D(G(z)): 0.0072 / 0.0078
[12/100][428/600] Loss_D: 0.0230 Loss_G: 1.0777 D(x): 0.9952 D(G(z)): 0.0177 / 0.0161
[12/100][429/600] Loss_D: 0.0119 Loss_G: 1.0978 D(x): 0.9959 D(G(z)): 0.0076 / 0.0067
[12/100][430/600] Loss_D: 0.0139 Loss_G: 1.0903 D(x): 0.9959 D(G(z)): 0.0097 / 0.0080
[12/100][431/600] Loss_D: 0.0147 Loss_G: 1.0922 D(x): 0.9956 D(G(z)): 0.0101 / 0.0079
[12/100][432/600] Loss_D: 0.0290 Loss_G: 1.0716 D(x): 0.9814 D(G(z)): 0.0073 / 0.0066
[12/100][433/600] Loss_D: 0.0179 Loss_G: 1.0804 D(x): 0.9885 D(G(z)): 0.0061 / 0.0060
[12/100][434/600] Loss_D: 0.0346 Loss_G: 1.0630 D(x): 0.9728 D(G(z)): 0.0060 / 0.0076
[12/100][435/600] Loss_D: 0.0132 Loss_G: 1.0922 D(x): 0.9933 D(G(z)): 0.0064 / 0.0078
[12/100][436/600] Loss_D: 0.0183 Loss_G: 1.0824 D(x): 0.9912 D(G(z)): 0.0092 / 0.0109
[12/100][437/600] Loss_D: 0.0249 Loss_G: 1.0730 D(x): 0.9887 D(G(z)): 0.0130 / 0.0147
[12/100][438/600] Loss_D: 0.0099 Loss_G: 1.1005 D(x): 

[12/100][522/600] Loss_D: 0.0177 Loss_G: 1.0840 D(x): 0.9912 D(G(z)): 0.0084 / 0.0085
[12/100][523/600] Loss_D: 0.0163 Loss_G: 1.0920 D(x): 0.9953 D(G(z)): 0.0113 / 0.0104
[12/100][524/600] Loss_D: 0.0216 Loss_G: 1.0781 D(x): 0.9907 D(G(z)): 0.0114 / 0.0102
[12/100][525/600] Loss_D: 0.0206 Loss_G: 1.0804 D(x): 0.9905 D(G(z)): 0.0106 / 0.0094
[12/100][526/600] Loss_D: 0.0137 Loss_G: 1.0965 D(x): 0.9949 D(G(z)): 0.0085 / 0.0072
[12/100][527/600] Loss_D: 0.0400 Loss_G: 1.0510 D(x): 0.9700 D(G(z)): 0.0073 / 0.0086
[12/100][528/600] Loss_D: 0.0269 Loss_G: 1.0664 D(x): 0.9892 D(G(z)): 0.0156 / 0.0170
[12/100][529/600] Loss_D: 0.0234 Loss_G: 1.0715 D(x): 0.9836 D(G(z)): 0.0062 / 0.0076
[12/100][530/600] Loss_D: 0.0204 Loss_G: 1.0735 D(x): 0.9903 D(G(z)): 0.0104 / 0.0123
[12/100][531/600] Loss_D: 0.0195 Loss_G: 1.0848 D(x): 0.9980 D(G(z)): 0.0172 / 0.0160
[12/100][532/600] Loss_D: 0.0246 Loss_G: 1.0757 D(x): 0.9913 D(G(z)): 0.0147 / 0.0124
[12/100][533/600] Loss_D: 0.0152 Loss_G: 1.0927 D(x): 